# Projet

## Collecte de données

In [1]:
!pip install sparqlwrapper
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get casino image
query = """SELECT ?image {
  ?data wdt:P31 wd:Q133215;
        wdt:P18 ?image
}
LIMIT 128"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["image"]["value"],
        )
    )

In [3]:
import pandas as pd
dataframe = pd.DataFrame(array, columns=["image"])
dataframe = dataframe.astype(
    dtype={"image": "<U200"}
)
dataframe

,image
0,http://commons.wikimedia.org/wiki/Special:File...
1,http://commons.wikimedia.org/wiki/Special:File...
2,http://commons.wikimedia.org/wiki/Special:File...
3,http://commons.wikimedia.org/wiki/Special:File...
4,http://commons.wikimedia.org/wiki/Special:File...
...,...
123,http://commons.wikimedia.org/wiki/Special:File...
124,http://commons.wikimedia.org/wiki/Special:File...
125,http://commons.wikimedia.org/wiki/Special:File...
126,http://commons.wikimedia.org/wiki/Special:File...


In [4]:
import os
try:
    os.mkdir('images')
except:
    pass

Nous allons écrire une méthode pour extraire les exifs

Nous allons maintenant télécharger les images

In [5]:
import requests
import shutil
import os
import json

count = 0
def download_image(url):
    global metadata, count
    if count % 10 == 0:
        print("Téléchargé ", count, " images") 
    count += 1

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        nom = os.path.basename(url)[-30:]
        chemin = "images/"+nom
        with open(chemin, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

In [6]:
print("Début du téléchargement")
dataframe.image.apply(download_image)

Début du téléchargement
Téléchargé  0  images
Téléchargé  10  images
Téléchargé  20  images
Téléchargé  30  images
Téléchargé  40  images
Téléchargé  50  images
Téléchargé  60  images
Téléchargé  70  images
Téléchargé  80  images
Téléchargé  90  images
Téléchargé  100  images
Téléchargé  110  images
Téléchargé  120  images


0      200
1      200
2      200
3      200
4      200
      ... 
123    200
124    200
125    200
126    200
127    200
Name: image, Length: 128, dtype: int64

On enregistre les exifs

In [8]:
!pip install scikit-learn
import os, sys, json
from PIL import Image
from PIL.ExifTags import TAGS
import numpy
import math
from sklearn.cluster import KMeans


def extraire_donnees(image_chemin):
    imgfile = Image.open(image_chemin)
    try:
        exif_data = imgfile._getexif()
    except:
        print(f"les EXIFs n'ont pas pu être extraits depuis {image_chemin}")
        exif_data = {}
    image_metadata = dict()
    if exif_data:
        for tag, value in exif_data.items():
            if(isinstance(value, bytes)):
                continue
            if (not isinstance(value, int)) and (not isinstance(value, float)):
                value = f"{value}"
            if tag in TAGS:
                image_metadata[TAGS[tag]] = value
    return image_metadata
print("Fonction extraction OK")

Defaulting to user installation because normal site-packages is not writeable
Fonction extraction OK


Extraction des couleurs

In [9]:
def extraire_couleurs(image_chemin):
    imgfile = Image.open(image_chemin)
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    print(image_chemin)
    clusters = KMeans(n_clusters=2, n_init=2)
    couleurs = []
    try :
        clusters.fit(numarray)
        for i in range(2):
            couleur = ("#%02x%02x%02x"
                % (
                    math.ceil(clusters.cluster_centers_[i][0]),
                    math.ceil(clusters.cluster_centers_[i][1]),
                    math.ceil(clusters.cluster_centers_[i][2]),
                ))
            couleurs.append(couleur)
    except:
        print(f"les couleurs n'ont pas pu être extraits depuis {image_chemin}")
        couleur = "#00000"
        for i in range(2):
            couleurs.append(couleur)
    print(couleurs)
    return couleurs

In [10]:
def ajout_tag(dat, filename, tags):
    dat[filename]['tags'] = tags
    return dat

def lire_entree(dat, filename):
    input_file = "user_input/"+filename+".txt"
    tagstr = ""
    if os.path.exists(input_file):
        with open(input_file, 'r') as fichier:
            tagstr = fichier.read()
    tags = [partie for partie in tagstr.split() if partie]
    return ajout_tag(dat, filename, tags)

Construction du JSON

In [11]:
json_dat = {}
for filename in os.listdir("images"):
    json_dat[filename] = {
        'exif': extraire_donnees("images/"+filename),
        'color': extraire_couleurs("images/"+filename)
    }
    json_dat = lire_entree(json_dat, filename)
print(json_dat)
json.dump(json_dat, open("img_data.json", "w"), indent=2)

images/%20Casino%2C%20Las%20Vegas.jpg
['#b2cae5', '#565254']
images/%20Foto%20postcard%201940s.jpg
['#4a4b4d', '#d4d5d6']
images/%20Rico%20%28IMG%202934%29.jpg
['#daded8', '#6f7565']
images/%20Station%20hotel%20tower.jpg
['#554c4b', '#b6b7d7']
images/%20Vegas%20December%202013.jpg
['#5f5448', '#b9bec2']
images/%20Vegas%20Wynn%20P4220706.jpg
['#4c5248', '#b0c4d6']
les EXIFs n'ont pas pu être extraits depuis images/%2C%20NV%20HAER%20NV-14-12.tif
images/%2C%20NV%20HAER%20NV-14-12.tif
les couleurs n'ont pas pu être extraits depuis images/%2C%20NV%20HAER%20NV-14-12.tif
['#00000', '#00000']
images/.%20Casino%20Gran%20Madrid.jpg
['#b5cae7', '#3d3e38']
images/.%20Lema%C3%AEtre%20Jobard.jpg
['#bfb6a1', '#6e634a']
images/0-%20Camuy%20Puerto%20Rico.jpg
['#93b6ca', '#5d544d']
images/0-%20Vivid%20Sydney%202015.jpg
['#84747e', '#232636']
images/02017%2003%20%28cropped%29.jpg
['#444342', '#97acc5']
images/0Charlie%27s%20Decatur%201.jpg
['#4a4137', '#ecedec']
images/0Hotel%20Casino%20Carrasco.jpg
['#2

Maintenant, on a les données à l'état brut. On va les transformer pour garder des données traitables pour l'analyse

In [1]:
"""
 single_picdata peut être un dictionnaire {exif, color, tags} ou {}
 La sortie doit être un tableau (ou vecteur) traitable pour l'analyse
"""
def transform_data(single_picdata):
    return []

On initialise ensuite un tableau avec nos données traitables pour 10 images aléatoires

In [2]:
from random import sample
import json, os

def read_pic_datas(picnames):
    with open('img_data.json', 'r') as f:
        pic_json_data = json.load(f)
    pic_json_keys = pic_json_data.keys()
    res_dat = []
    for picname in picnames:
        single_picdat = pic_json_data[picname] if picname in pic_json_keys else {}
        res_dat.append(transform_data(single_picdat))
    return res_dat

pics = sample(os.listdir("images"), 10)
picdata_matrix = read_pic_datas(pics)

On demande les préférences utilisateur sur ces 10 images

In [3]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output
from threading import Semaphore

output = widgets.Output()

liked = Semaphore(0)
shown_image_idx = 0
likes = [None] * len(pics)

def likeit(_b):
    global shown_image_idx, liked
    print("Like")
    likes[shown_image_idx] = True
    liked.release()

def hateit(_b):
    global shown_image_idx, liked
    print("Hate")
    likes[shown_image_idx] = False
    liked.release()

like_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Like',
    icon='heart'
)
like_button.style = {'button_color': '#ff4757'}
like_button.on_click(likeit)

hate_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Dislike',
    icon='thumbs-down'
)
hate_button.style = {'button_color': '#5352ed'}
hate_button.on_click(hateit)
buttons = widgets.HBox([like_button, hate_button])

def show_with_btn(nomfichier):
    global shown_image_idx, output
    with open("images/"+nomfichier, 'rb') as f:
        shown_image = f.read()
    picwidget = widgets.Image(
        value=shown_image,
    )
    clear_output()
    main = widgets.VBox([buttons, picwidget])
    display(main,output)

shown_image_idx = 0
for pic in [pics[0]]:
    show_with_btn(pic)
    shown_image_idx += 1


In [ ]:
from ipywidgets import GridspecLayout, Image, interact
paths = []

checkboxes = [widgets.Checkbox(value=False, description='Favorite') for _ in range(len(pics))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=2, n_rows=len(pics), width='400px')
for i, (img, checkbox) in enumerate(zip(pics, checkboxes)):
  file = open("images/"+img, "rb")
  image = file.read()
  image_widget = widgets.Image(
    value=image,
    format='png',
    width=100,
    height=100,
  )
  layout[i,0] = image_widget
  layout[i, 1] = checkbox

button = widgets.Button(description="Select")
output = widgets.Output()

likes = [None]*len(pics)

def get_selected_images(btn):
    global paths, likes
    paths = []
    for i, checkbox in enumerate(checkboxes):
        likes[i] = checkbox.value
    with output:
        output.clear_output()
        print("Likes vector :")
        print(likes)

button.on_click(get_selected_images)

display(layout, button, output)